## Лабораторная работа 7. Бустинги!

В этой домашке познакомимся с новым методом ансамблирования — бустингом, а также немножко доработаем фичи для предыдущего датасета, чтобы получать лучшее качество на тестовом датасете.

Контест https://contest.yandex.ru/contest/40410

#### Дискуссия
Краеугольными камнями машинлернера являются данные и модели: применили более качественную модель — получили прирост метрик, нашли дополнительные данные или сгенерировали информативные признаки, которые наша модель не может сгенерировать сама — получили прирост метрик. Поскольку мы продолжаем тему ансамблирования, то разберемся с ультимативным методом ансамблирования — бустингом, который до сих пор дает топовые места в табличных соревнованиях и используется в качестве модели верхнего уровня в сложных стэкинговых пайплайнах Яндекса.

#### Адаптивный бустинг
Первой задачей будет имплементация алгоритма SAMME.R для адаптивного бустинга — последовательного обучения слабых классификаторов с **перевзвешенными** объектами обучающей выборки для уточнения предсказаний именно на примерах с сильно отличающимися от истинных предсказаниями.

Наша имплементация будет опираться на авторскую монографию ([ссылка](https://hastie.su.domains/Papers/samme.pdf)).

Прочтите оттуда описание алгоритма 4 и используйте следующие подсказки для полноценной имплементации:
* В качестве базового классификатора применяйте (и держите в голове при написании кода) `DecisionTreeClassifier` из `sklearn`, т.к. он умеет работать со взвешенными объектами обучающей выборки. В качестве бонусного задания подумайте, как модифицировать нашу реализацию деревьев, чтобы она тоже умела работать с приоритетными объектами. Однако, заметьте, что бустить этим алгоритмом можно не только деревья, но и другие модели, которые могут предсказывать вероятности, например, логистическую регрессию, перцептрон и другие.
* Заметьте, что в статье истинные метки классов пересчитаны некоторым нестандартным образом (формула (2) похожа на OneHot, но сглаженный), там это мотивировано вероятностной постановкой задачи. На практике так часто делают, чтобы улучшить протекание градиента в нейросетях или распространение информации в весах бустинга, поскольку неверные предикты не зануляются совсем при пересчете весов.
* При подсчете вероятности класса для данного объекта берется сумма скоров, предсказаных базовыми классификаторами, потом перевзвешивается при помощи `softmax`, то есть формула для вероятности принимает вид (подумайте, чему эта формула соответствует при подстановке $h$ через вероятность):
$$
    P(y_i = k|x_i) = \frac{\exp\left\{\frac{1}{K-1}\sum_{m=1}^Mh_k^{(m)}(x_i)\right\}}{\sum_{k'=1}^K\exp\left\{\frac{1}{K-1}\sum_{m=1}^Mh_{k'}^{(m)}(x_i)\right\}}
$$

**1. (1 балл) Сдайте реализацию перевзвешивания в адаптивном бустинге в контест.**

**2. (2 балла) Сдайте реализацию обучения классификатора в контест.**

**3. (2 балла) Сдайте реализацию предсказания в адаптивном бустинге в контест.**

In [ ]:
from src import MyAdaBoostClassifier

Воспользуемся полюбившейся библиотекой, чтобы смотреть на предсказания

In [ ]:
!pip install mlxtend

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import make_moons


def make_sunny_moons(n_sun=50, n_moons=100, noise=0.0, sun_radius=1.9, theta=None):
    X_moons, y_moons = make_moons(n_samples=n_moons, noise=noise, random_state=0xC0FFEE)
    if not n_sun:
        return X_moons, y_moons

    rng = np.random.default_rng(0xC0FFEE)
    angles = np.arange(0, 2 * np.pi, 2 * np.pi / n_sun)
    X_sun = sun_radius * np.column_stack([np.cos(angles), np.sin(angles)]) + np.array([0.5, 0.25])
    X_sun += rng.normal(scale=noise, size=X_sun.shape)
    y_sun = 2 * np.ones(n_sun)

    X = np.vstack([X_moons, X_sun])
    y = np.concatenate([y_moons, y_sun]).astype(int)
    X -= X.mean(axis=0)

    if theta is None:
        theta = np.pi / 4
    c, s = np.cos(theta), np.sin(theta)
    R = np.array(((c, -s), (s, c)))
    X = X @ R

    return X, y


X, y = make_sunny_moons(n_sun=150, n_moons=300, noise=0.15)
_ = plt.figure(figsize=(6, 6))
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='cool')
plt.show();

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from mlxtend.plotting import plot_decision_regions
from sklearn.metrics import accuracy_score


def make_clf_plot(classifier, axis):
    classifier.fit(X, y)
    plot_decision_regions(X, y, clf=classifier, legend=2, ax=axis)
    accuracy = accuracy_score(y, classifier.predict(X))
    axis.set_title(f"{classifier.__class__.__name__}, accuracy = {accuracy:2.2f}")


_ = plt.figure(figsize=(6, 6))
axis = plt.axes()
clf = MyAdaBoostClassifier(n_estimators=10, base_estimator=DecisionTreeClassifier, max_depth=3, seed=42)
make_clf_plot(clf, axis)

Теперь посмотрим на процесс обучения

In [ ]:
plt.plot(clf.error_history)
plt.title('Weighted errors for sequentially learned estimators')
plt.xlabel('estimator index')
plt.ylabel('weighted error')
plt.show()
print(f'Last estimator weighted error is {clf.error_history[-1]:0.3f}')

Видим, что ошибка последнего близка к нулю. Предположите, как будет изменяться ошибка при добавлении большего числа базовых классификаторов? \
Давайте теперь проверим ваше предположение!

In [ ]:
clf = MyAdaBoostClassifier(n_estimators=20, base_estimator=DecisionTreeClassifier, max_depth=3, seed=42)
clf.fit(X, y)
plt.plot(clf.error_history)
plt.title('Weighted errors for sequentially learned estimators')
plt.xlabel('estimator index')
plt.ylabel('weighted error')
plt.show()
print(f'Last estimator weighted error is {clf.error_history[-1]:0.3f}')

Попытайтесь объяснить, почему так происходит?

<details>

  <summary><b>Нажмите однократно, чтобы раскрыть</b></summary>

Поскольку это взвешенная ошибка последнего классификатора, как только мы выучиваем какую-то группу тяжелых примеров, они начинают предсказываться корректно и больший вклад начинают делать примеры, которым мы на данном шаге не уделили внимания. В целом error_rate слабых классификаторов и не должен сходится к нулю, но их ансамбль работает на порядок лучше из-за разнообразия, что мы и видели выше на близкой к 1 accuracy.

</details>

Сравните такой взвешенный ансамбль деревьев со случайным лесом из `sklearn`

In [ ]:
from sklearn.ensemble import RandomForestClassifier

fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(13, 6.5))

for classifier, axis in zip(
        (
            # put models (Boosting and Forest) with equivalent parameters for trees (e.g. max_depth=3/4, etc)
            # see example with GaussianNB model below 
        ), axes):
    make_clf_plot(classifier, axis)

Видим, что несмотря на одинаковое количество разделяющих сегментов (поскольку количество и глубина деревьев зафиксированы), бустинг справляется с классификацией сильно лучше, особенно на сложных примерах, где данные нелинейны и находятся близко друг к другу. Однако такая гибкость может привести к переобучению:

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(13, 6.5))

for classifier, axis in zip(
        (
            # put here both models with many estimators
        ), axes):
    make_clf_plot(classifier, axis)

В зависимости от сида и удачи, вы можете поймать моменты, когда адаптивный бустинг рисует островки одного класса внутри региона другого, что является одним из хороших признаков переобучения, причем все это на трейне! Поэтому число эстиматоров надо тюнить и смотреть на метрики на тесте.

Теперь попробуем забустить логистическую регрессию, чтобы порадоваться какой мощный метод мы заимплементировали.

In [ ]:
from sklearn.linear_model import LogisticRegression

fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(13, 6.5))

for classifier, axis in zip(
        (
            # Boost logistic regression and plot alongside with just one logreg, regularize it slightly
        ), axes):
    make_clf_plot(classifier, axis)

Видим, что качество очень далеко от ожидаемого, что же случилось?

<details>

  <summary><b>Нажмите однократно, чтобы раскрыть</b></summary>

Поскольку предсказание формируется из линейной комбинации скоров базовых моделей (см формулу выше), мы все равно получаем линейную модель, которая не может разделить нелинейную выборку и профита от бустинга нет..

</details>


Применим другую модель послабее, зато нелинейную, которая предсказывает гауссовы центроиды

In [ ]:
from sklearn.naive_bayes import GaussianNB

fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(13, 6.5))

for classifier, axis in zip(
        (
                MyAdaBoostClassifier(n_estimators=30, base_estimator=GaussianNB, seed=42),
                GaussianNB()

        ), axes):
    make_clf_plot(classifier, axis)

Отлично, с адаптивным бустингом разобрались, теперь давайте имплементируем знаменитый градиентный бустинг.

#### Градиентный бустинг
Идея градиентного бустинга заключается в том, чтобы выполнить дискретную градиентную оптимизацию какой-нибудь выпуклой функции потерь, предсказывая на каждом шагу градиент специально обученным **регрессором**. Другая точка зрения состоит в том, что каждая последующая модель бустинга учится предсказывать отклонение ансамбля предыдущих от правильного значения, так что они постепенно уточняют итоговый предикт. \
Алгоритм градиентного бустинга можно прочитать в [учебнике](https://ml-handbook.ru/chapters/grad_boost/intro). Наша реализация будет иметь следующие особенности:
* Поскольку модели учатся предсказывать градиент, то вместо базовых классификаторов нам потребуются базовые **регрессоры**, мы используем регрессионные деревья из `sklearn`
* Константный лернинг рейт (гамма) для борьбы с переобучением и упрощения реализации
* Будет реализована только бинарная классификация, поскольку градиент по предсказаниям модели в случае многоклассовой классификации — это вектор по числу классов, нам потребуется `К` регрессионных деревьев на каждом шаге (каждое из них умеет предсказывать только одно число). Держите это в голове, когда используете градиентный бустинг из `sklearn`: в нем число деревьев, из которых состоит модель, на самом деле равно число_классов * `n_estimators`. Для бинарного случая такой проблемы нет, поскольку достаточного одного числа для предсказания бинарной вероятности
* Наша имплементация опирается на логиты — сырые предсказания регрессионной модели, принимающие вещественные значения. Вероятности из них по аналогии с логистической регрессией будем получать при помощи сигмоиды
* В качестве функции потерь возьмем отрицательное лог-правдоподобие. Объединяя с пунктом выше имеем такие формулы:
$$
    \text{model}(x) = h, \ \text{probability} = \sigma(h) \\
    \mathcal{L}(\text{model}, X, Y) = -\sum_{(x, y) \in X\oplus Y} y \cdot\log(\text{probability}(\text{model}(x))) + (1 - y) \cdot \log(1 - \text{probability}(\text{model}(x)))
$$

**4. (1 балл) Сдайте реализацию функции потерь в градиентном бустинге в контест.**

**5. (2 балла) Сдайте реализацию обучения классификатора в контест.**

**6. (2 балла) Сдайте реализацию предсказания в градиентном бустинге в контест.**

In [ ]:
from src import MyBinaryTreeGradientBoostingClassifier

In [ ]:
X, y = make_moons(n_samples=300, noise=0.15, random_state=42)


def make_clf_plot(classifier, axis):
    classifier.fit(X, y)
    plot_decision_regions(X, y, clf=classifier, legend=2, ax=axis)
    accuracy = accuracy_score(y, classifier.predict(X))
    axis.set_title(f"{classifier.__class__.__name__}, accuracy = {accuracy:2.2f}")


_ = plt.figure(figsize=(6, 6))
axis = plt.axes()
clf = MyBinaryTreeGradientBoostingClassifier(n_estimators=10, learning_rate=1.0, seed=42, max_depth=4)
make_clf_plot(clf, axis)

Посмотрим на перфоманс нашей модели с  точки зрения функции потерь (должна глобально убывать)

In [ ]:
plt.plot(clf.loss_history)
plt.title('LogLoss for sequentially learned estimators')
plt.xlabel('estimator index')
plt.ylabel('negative log-likelihood')
plt.show();

Пронаблюдаем зависимость обучения и переобучения от лернинг рейта

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows=3, figsize=(6 * 2 + 1, 6 * 3 + 1))

def plot_loss(classifier, axis):
    axis.plot(classifier.loss_history)
    axis.set_title(f'LogLoss for sequentially learned estimators for lr={classifier.learning_rate}')
    axis.set_xlabel('estimator index')
    axis.set_ylabel('negative log-likelihood')

for classifier, axis in zip(
        (
            # put three different learning rates
        ), axes):
    make_clf_plot(classifier, axis[0])
    plot_loss(classifier, axis[1])

Видим, что модель почти не переобучается, и даже при плохих значениях лернинг рейта пытается что-то предсказать. Что будет, если менять число эстиматоров?

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows=3, figsize=(6 * 2 + 1, 6 * 3 + 1))

for classifier, axis in zip(
        (
            # put three different estimator counts
        ), axes):
    make_clf_plot(classifier, axis[0])
    plot_loss(classifier, axis[1])

Видим, что есть некоторый трейд-офф между величиной градиентного шага и количеством таких шагов (что очень похоже на случай линейных моделей). В целом, поскольку шаг бустинга при больших деревьях может выполняться долго, есть смысл оставлять число эстиматоров разумным и тюнить лернинг рейт, который не влияет на скорость работы.

Насколько предикторы в градиентном бустинге равны между собой (такое любят спрашивать на собеседованиях). Давайте испортим самую первую и самую последнюю базовую модель в наших классификаторах. Подумайте, как в такой ситуации поведет себя адаптивный бустинг.

In [ ]:
class ModelPredictingNoise:
    def predict(self, X):
        ...
        return random_predictions
    def predict_proba(self, X):
        ...
        return random_noise

fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(6 * 2 + 1, 6 + 1))
for classifier, axis, spoil_id in zip(
        (
                MyBinaryTreeGradientBoostingClassifier(n_estimators=5, learning_rate=1.0, seed=42, max_depth=4),
                MyBinaryTreeGradientBoostingClassifier(n_estimators=5, learning_rate=1.0, seed=42, max_depth=4)
        ), axes, [0, -1]):
    classifier.fit(X, y)
    classifier.estimators[spoil_id] = ModelPredictingNoise()
    plot_decision_regions(X, y, clf=classifier, legend=2, ax=axis)
    accuracy = accuracy_score(y, classifier.predict(X))
    axis.set_title(f"{classifier.__class__.__name__}, accuracy = {accuracy:2.2f}")

Видим, что при достаточно больших значениях лернинг рейта, уточняющие модели дают ощутимый вклад, но все равно, первое дерево дает более весомый вклад в ответ.

#### Теперь в лабе можно пользоваться лучшей общедоступной реализацией бустинга CatBoost, ура :)

In [ ]:
!pip install catboost

Вернемся к нашей задаче с классификацией музыки

In [ ]:
!kaggle datasets download -d purumalgi/music-genre-classification

In [ ]:
!unzip music-genre-classification.zip -d ./data

In [ ]:
import pandas as pd

train_csv = pd.read_csv('./data/train.csv')
test_csv = pd.read_csv('./data/test.csv')
submission_csv = pd.read_csv('./data/submission.csv')

In [ ]:
train_csv.head()

In [ ]:
test_csv.head()

In [ ]:
submission_csv.head()

В прошлый раз мы пытались предсказать жанр музыки основываясь только на вещественных фичах, которые способны были скушать наши деревья, но поскольку мы обладаем теперь более мощными моделями, давайте сделаем более мощные фичи. Тем более что это очень просто с катбустом.

Напомним наш прошлый топовый пайплайн без учета стэкинга. **Положите рядом с ноутбуком пикл индексов для разбиения из прошлой домашки, а также логи хэндлера экспериментов, чтобы продолжить существующий лидерборд.**

In [ ]:
import pickle
import os
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold

filename = 'indices.pckl'
if os.path.exists(filename):
    with open(filename, 'rb') as f:
        indices = pickle.load(f)
else:
    indices = {}
    indices['train_indices'], indices['test_indices'] = train_test_split(
        np.arange(len(train_csv)),
        test_size=2996,
        stratify=train_csv['Class'],
        shuffle=True,
        random_state=0xBA0BAB
    )

    train_df = train_csv.iloc[indices['train_indices']]
    cv_splitter = StratifiedKFold(
        n_splits=3,
        shuffle=True,
        random_state=0xBED
    )
    indices['cv_iterable'] = []
    for train_indices, val_indices in cv_splitter.split(train_df.drop('Class', axis=1), train_df['Class']):
        indices['cv_iterable'].append(
            (train_indices, val_indices)
        )
    with open(filename, 'wb+') as f:
        pickle.dump(indices, f)

In [ ]:
train_indices = indices['train_indices']
test_indices = indices['test_indices']
cv_iterable = indices['cv_iterable']
X_train = train_csv.iloc[train_indices].drop('Class', axis=1)
y_train = train_csv.iloc[train_indices]['Class']
X_test = train_csv.iloc[test_indices].drop('Class', axis=1)
y_test = train_csv.iloc[test_indices]['Class']

In [ ]:
from src import Logger, ExperimentHandler

logger = Logger('./logs')
scorer = ExperimentHandler(
    X_train, y_train, X_test, y_test, cv_iterable, logger,
    metrics={
        'BalancedAccuracy': 'balanced_accuracy',
        'NegLogLoss': 'neg_log_loss'
    }
)

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

forest_pipeline = Pipeline(
    [
        (
            'extract numeric features',
            ColumnTransformer(
                [
                    (
                        'drop words',
                        'drop',
                        ['Artist Name', 'Track Name']
                    )
                ],
                remainder='passthrough'
            )
        ),
        (
            'fill missing values',
            SimpleImputer(strategy='constant', fill_value=X_train.apply(pd.to_numeric, errors='coerce').max().max())
        ),
        (
            'estimator',
            RandomForestClassifier(n_estimators=200, random_state=0x5EED)
        )
    ]
)

Попробуем в нашем пайплайне применить бустинг и померить профит

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gradboost_pipeline = Pipeline(
    [
        (
            'extract numeric features',
            ColumnTransformer(
                [
                    (
                        'drop words',
                        'drop',
                        ['Artist Name', 'Track Name']
                    )
                ],
                remainder='passthrough'
            )
        ),
        (
            'fill missing values',
            SimpleImputer(strategy='constant', fill_value=X_train.apply(pd.to_numeric, errors='coerce').max().max())
        ),
        (
            'estimator',
            GradientBoostingClassifier(n_estimators=50, random_state=0x5EED)
            # это 50 * 11 деревьев, тк у нас 11 классов
        )
    ]
)

In [ ]:
scorer.logger.leaderboard

In [ ]:
scorer.run(gradboost_pipeline, name='gradboost_on_old_features')

In [ ]:
from catboost import CatBoostClassifier

catboost_pipeline = Pipeline(
    [
        (
            'extract numeric features',
            ColumnTransformer(
                [
                    (
                        'drop words',
                        'drop',
                        ['Artist Name', 'Track Name']
                    )
                ],
                remainder='passthrough'
            )
        ),
        (
            'fill missing values',
            SimpleImputer(strategy='constant', fill_value=X_train.apply(pd.to_numeric, errors='coerce').max().max())
        ),
        (
            'estimator',
            CatBoostClassifier(iterations=200, random_state=0x5EED)
        )
    ]
)

In [ ]:
scorer.run(catboost_pipeline, name='catboost_on_old_features')

(Катбуст пока не побеждает)

Видим, что в нашем пайплайне мы отказались от важного набора фичей — текстовых фичей. Почему они могут нам докинуть качества:

In [ ]:
intersection_of_artists = # найдите пересечение между артистами из терйна и теста
print("Example of overlap between artist names from train and test")
print(list(intersection_of_artists)[:10])
print('Count of overlapping artists:', len(intersection_of_artists))

Проверим предположение, что исполнители в среднем играют музыку в одном и том же жанре

In [ ]:
track_genre_per_artist_counts = X_train.join(y_train)[['Artist Name', 'Class']].groupby(['Artist Name', 'Class']).agg(
    count_col=pd.NamedAgg(column="Class", aggfunc="count")
)
track_genre_per_artist_counts.head()

Посмотрим, сколько артистов с более чем одним жанром, сколько у них трэков.

In [ ]:
print('Count of unique artists:', X_train['Artist Name'].nunique())
print('Count of artists with more than two genres:',
      (track_genre_per_artist_counts.groupby(level=0).count() > 1).sum().item())
top_represented_tracks_count = track_genre_per_artist_counts.groupby(level=0)['count_col'].max()
print('Mean value of top-repesented genre tracks count:', top_represented_tracks_count.mean())
print('Max value of top-repesented genre tracks count:', top_represented_tracks_count.max())
plt.bar(x=np.arange(top_represented_tracks_count.shape[0]),
        height=top_represented_tracks_count.sort_values(ascending=False))
plt.show();

Интуитивно кажется, что примеров одножанровых артистов не так много, особенно, учитывая, что оверлап с тестом не такой большой. Но они есть и надо засунуть их в модель, чтобы понять какой профит будет от этого. Только вопрос, как это сделать. Если, допустим, трактовать имя исполнителя, как категориальную фичу, то ординальность получится более 8к, а значит никакого one-hot кодирования сделать не получится. У нас будет очень разреженная фича, задача получится переопределенной и потребуется модель с большой "емкостью", чтобы запомнить, какой категории соответствует какой таргет. В этом месте у нас два пути: работать с фичей как с текстовой или использовать другие способы кодирования, научимся делать оба!

Первый вариант, закодировать таргетом, добавив для теста вариант фичи "не встречалась в трейне", напишите соответствующий кодировщик средствами `sklearn`

In [ ]:
artist_top_genre_tuples = track_genre_per_artist_counts.groupby(level=0).idxmax()['count_col'].to_list()

In [ ]:
dict(artist_top_genre_tuples)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin


class SimpleTargetEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.dict_of_artists = None

    def fit(self, X, y):
        X = pd.DataFrame(X, columns=X_train.columns)
        if isinstance(y, pd.Series):
            X['Class'] = y.values
        elif isinstance(y, np.ndarray):
            X['Class'] = y
        track_genre_per_artist_counts = # подсчитайте сколько треков данного артиста принадлежат данному жанру
        artist_top_genre_tuples = # постройте словарь для реплейса
        self.dict_of_artists = dict(artist_top_genre_tuples)
        return self

    def transform(self, X):
        X = pd.DataFrame(X, columns=X_train.columns)
        X['Target Encoded Artists'] = X['Artist Name'].map(self.dict_of_artists).fillna(value=-1).astype(int)
        return X

In [ ]:
tr = SimpleTargetEncoder()
tr.fit(X_train, y_train).transform(X_train).head()

In [ ]:
tr.transform(X_test).head()

Теперь такую категориальную фичу можно преобразовать в OneHot и использовать в пайлпайне

In [ ]:
from sklearn.preprocessing import OneHotEncoder

catboost_pipeline = Pipeline(
    [
        (
            'target encode',
            ColumnTransformer(
                [
                    (
                        'Target Encode Artist Name',
                        SimpleTargetEncoder(),
                        X_train.columns
                    )
                ],
            )
        ),
        (
            'extract numeric features',
            ColumnTransformer(
                [
                    (
                        'OHE for target encoded artist names',
                        OneHotEncoder(sparse=False, handle_unknown='ignore'),
                        [16, ] # those magic numbers are used to overcome issue with improper handling of dataframes by column transformer
                    ),
                    (
                        'drop words',
                        'drop',
                        [0, 1]  # those magic numbers are used to overcome issue with improper handling of dataframes by column transformer
                    )
                ],
                remainder='passthrough'
            )
        ),
        (
            'fill missing values',
            SimpleImputer(strategy='constant', fill_value=X_train.apply(pd.to_numeric, errors='coerce').max().max())
        ),
        (
            'estimator',
            CatBoostClassifier(iterations=200, random_state=0x5EED)
        )
    ]
)

In [ ]:
scorer.run(catboost_pipeline, name='catboost_on_simple_target_encoded_features')

Стало сильно лучше!

In [ ]:
scorer.logger.leaderboard.sort_values(by='BalancedAccuracy_test', ascending=False)

На самом деле нет необходимости кодировать текстовые фичи мануально — ведь катбуст может работать с текстовыми из коробки, для этого он применяет Bag of Words/TF-IDF методы. Поскольку ключевые слова из названий исполнителей и песен могут нести положительный сигнал, мы ожидаем прирост качества от этого подхода. А еще, как следует из названия, он поддерживает категориальные фичи множеством разных методов их кодирования. Воспользуйтесь этим и постройте сильную модель.

In [ ]:
class PandasSimpleImputer(SimpleImputer):
    """
    A wrapper around `SimpleImputer` to return data frames with columns.
    """

    def fit(self, X, y=None):
        self.columns = X.columns
        return super().fit(X, y)

    def transform(self, X):
        return pd.DataFrame(super().transform(X), columns=self.columns)

strong_catboost_pipeline = Pipeline(
    [
        (
            'fill missing values',
            PandasSimpleImputer(strategy='constant', fill_value=X_train.apply(pd.to_numeric, errors='coerce').max().max())
        ),
        (
            'estimator',
            CatBoostClassifier(SOMETHING) # разберитесь в параметрах катбуста, чтобы он обучался на текстовых и категориальных фичах
        )
    ]
)

In [ ]:
strong_catboost_pipeline.fit(X_train, y_train)

Проверим только качество на тесте, чтобы не ждать, когда катбуст обучится на всех фолдах.

In [ ]:
from sklearn.metrics import balanced_accuracy_score
predictions = strong_catboost_pipeline.predict(X_test)
print('strong catboost balanced accuracy on test:', balanced_accuracy_score(y_test, predictions))
print('top balanced accuracy from our previous results is:', scorer.logger.leaderboard['BalancedAccuracy_test'].max())

Отрефлексируйте полученный результат.